In [1]:
import numpy as np
import pandas as pd
import pycountry

books = pd.read_csv("BX-Books.csv", encoding="ISO-8859-15")
users = pd.read_csv("BX-Users.csv")
users


,User-ID,User-City,User-State,User-Country,User-Age
0,8,timmins,ontario,"canada""",NaN
1,9,germantown,tennessee,"usa""",NaN
2,16,albuquerque,new mexico,"usa""",NaN
3,17,chesapeake,virginia,"usa""",NaN
4,19,weston,,NaN,"14"""
...,...,...,...,...,...
48294,278846,toronto,ontario,"canada""",23
48295,278849,georgetown,ontario,"canada""",23
48296,278851,dallas,texas,"usa""",33
48297,278852,brisbane,queensland,"australia""",32


In [37]:
def isValidISBN(isbn) -> bool:
    digits = str(isbn)
    # print(digits)
    if len(digits) == 10:
        sumCheck = 0
        for index, digit in enumerate(digits):
            sumCheck += (10 if digit.lower() == 'x' else int(digit)) * (10-index)
        return sumCheck % 11 == 0
    return False
books["ValidISBN"] = books.apply(lambda row: isValidISBN(row["ISBN"]),axis=1)


In [38]:
import regex as re
import numpy as np
users = pd.read_csv("BX-Users.csv")

users["User-Age"] = users.apply(
    lambda row: re.sub(r'\"$', "", row["User-Age"]) if not pd.isna(row["User-Age"]) else row["User-Age"],
    axis=1
)

users["User-Age"].fillna(users["User-Age"].median(), inplace=True)
users.astype({
    "User-Age": "int32",

})

def cleanNoise(s):
    s = re.sub(r'^\s*', "", s)
    s = re.sub(r'(\>|\.|\")+$', "", s)
    return s
    
users["User-Country"] = (users['User-Country']
                         .apply(lambda country: cleanNoise(country) if not pd.isna(country) else pd.NA))
users["User-Country"] = users['User-Country'].apply(
    lambda userCountry: pd.NA if not pd.isna(userCountry) and len(userCountry) < 3 else userCountry
)
users["User-Country"].unique()


array(['canada', 'usa', <NA>, 'united kingdom', 'philippines', 'india',
       'germany', 'quit', 'finland', 'spain', 'italy', 'france',
       'austria', 'netherlands', 'switzerland', 'new zealand', 'malaysia',
       'australia', 'portugal', 'singapore', 'taiwan', 'south africa',
       'alderney', 'latvia', 'sweden', 'ireland', 'iran', 'slovenia',
       'csa', 'hong kong', 'israel', 'denmark', 'japan', 'romania',
       'argentina', 'mexico', 'malta', 'trinidad and tobago', 'thailand',
       'brazil', 'guatemala', 'china', 'chile', 'bermuda', 'england',
       'jersey', 'south korea', 'greece', 'belgium', 'n/a', 'turkey',
       'u.s. of a', 'catalunya', 'moldova', 'czech republic', 'nigeria',
       'tanzania', 'tdzimi', 'bahamas', 'u.s.a', 'palau', 'uruguay',
       'luxembourg', 'egypt', 'slovakia', 'iceland', 'kuwait',
       'saint lucia', 'bulgaria', 'norway', 'costa rica', 'antarctica',
       'wales', 'united arab emirates', 'papua new guinea', 'colombia',
       'algeria'

In [39]:
import pycountry as pc
import pprint
matchCountryMap = {}
naCountries = {}
countries = pd.Series(users["User-Country"].unique())
naCount = 0
possibleNANames = ["n/a", "na"]
notFoundCountries = []
for country in countries:
    if not pd.isna(country):
        if not country.lower() in possibleNANames:
            countryName = str(country)
            countryName = countryName.title()
            try:
                if "." in countryName:
                    countryName = countryName.replace(".","").upper()
                matchCountryMap[country] = pc.countries.lookup(countryName)
            except LookupError as e:
                naCount += 1
                try:
                    matchCountryMap[country] = pc.countries.search_fuzzy(countryName)[0]
                except LookupError as _:
                    notFoundCountries.append(countryName)
                    print("Not found ", countryName)
        else:
            matchCountryMap[country] = pd.NA
pprint.pp(matchCountryMap)
pprint.pp(notFoundCountries)

Not found  Alderney
SubdivisionHierarchy(code='GB-ENG', country_code='GB', name='England', parent_code=None, type='Country')
Not found  Turkey
Not found  US OF A
Not found  Tdzimi
Not found  España
SubdivisionHierarchy(code='IT-FE', country_code='IT', name='Ferrara', parent='45', parent_code='IT-45', type='Province')
Not found  Lkjlj
Not found  N/A - On The Road
SubdivisionHierarchy(code='ES-PV', country_code='ES', name='Euskal Herria', parent_code=None, type='Autonomous community')
Not found  Yugoslavia
Not found  Richmond Country
Not found  Framingham
Not found  United Kindgonm
Not found  Universe
Not found  ST HELENA
SubdivisionHierarchy(code='GB-SCT', country_code='GB', name='Scotland', parent_code=None, type='Country')
Not found  Aroostook
Not found  Burma
Not found  Van Wert
Not found  Far Away
Not found  K1C7B1
Not found  Rutherford
SubdivisionHierarchy(code='GB-CMD', country_code='GB', name='Camden', parent='GB-ENG', parent_code='GB-GB-ENG', type='London borough')
Not found  UA

In [40]:
import pycountry as pc
# Manually match the country that pycountry failed to match
manualMatchCountryMap = {
    'Alderney': pc.countries.lookup("guernsey"),
    'Turkey': pc.countries.lookup("TUR"),
    'US OF A': pc.countries.lookup("USA"),
    'Tdzimi': pd.NA,
    'España': pc.countries.lookup("ESP"),
    'Lkjlj': pd.NA,
    'N/A - On The Road': pd.NA,
    'Yugoslavia': pd.NA,
    'Richmond Country': pd.NA,
    'Framingham' : pc.countries.lookup("USA"),
    'United Kindgonm': pc.countries.lookup("GB"),
    'Universe': pd.NA,
    'ST HELENA': pc.countries.lookup("SH"),
    'Aroostook': pc.countries.lookup("US"),
    'Burma': pc.countries.lookup("MM"),
    'Van Wert': pc.countries.lookup("USA"),
    'Far Away': pd.NA,
    'K1C7B1': pd.NA,
    'Rutherford': pd.NA,
    'UAE': pc.countries.lookup("ARE"),
    'Petrolwar Nation': pd.NA,
    'Shelby': pd.NA,
    'Catalonia': pc.countries.lookup("ESP"),
    'Caribbean Sea': pd.NA,
    'Cananda': pc.countries.lookup("CA"),
    'Pender': pd.NA,
    'La France': pc.countries.lookup("FR"),
    'Phillipines': pc.countries.lookup("PHL"),
    'Los Estados Unidos De Norte America': pc.countries.lookup("USA"),
    'Orange Co': pc.countries.lookup("USA"),
    'Unknown': pd.NA,
    'L`Italia': pc.countries.lookup("IT"),
    'Kern': pd.NA,
    'STTHOMASI': pd.NA,
    'Everywhere And Anywhere': pd.NA,
    'United Staes': pc.countries.lookup("USA"),
    'Deutschland': pc.countries.lookup("DE"),
}

In [41]:
for key in matchCountryMap:
    country = matchCountryMap[key]
    print(type(country))

<class 'pycountry.db.Country'>
<class 'pycountry.db.Country'>
<class 'pycountry.db.Country'>
<class 'pycountry.db.Country'>
<class 'pycountry.db.Country'>
<class 'pycountry.db.Country'>
<class 'pycountry.db.Country'>
<class 'pycountry.db.Country'>
<class 'pycountry.db.Country'>
<class 'pycountry.db.Country'>
<class 'pycountry.db.Country'>
<class 'pycountry.db.Country'>
<class 'pycountry.db.Country'>
<class 'pycountry.db.Country'>
<class 'pycountry.db.Country'>
<class 'pycountry.db.Country'>
<class 'pycountry.db.Country'>
<class 'pycountry.db.Country'>
<class 'pycountry.db.Country'>
<class 'pycountry.db.Country'>
<class 'pycountry.db.Country'>
<class 'pycountry.db.Country'>
<class 'pycountry.db.Country'>
<class 'pycountry.db.Country'>
<class 'pycountry.db.Country'>
<class 'pycountry.db.Country'>
<class 'pycountry.db.Country'>
<class 'pycountry.db.Country'>
<class 'pycountry.db.Country'>
<class 'pycountry.db.Country'>
<class 'pycountry.db.Country'>
<class 'pycountry.db.Country'>
<class '

# Match the country in the dataset

In [42]:
def matchCountry(name):
    if not pd.isna(name):
        country = 0
        if name in matchCountryMap:
            country = matchCountryMap.get(name)
        else:
            country = manualMatchCountryMap.get(name)
        if country is pd.NA or country is None: return "others"
        return country.name
    return "others"
# 
users["User-Country"] = users["User-Country"].apply(
    lambda name: matchCountry(name)
)
users["User-Country"]

0               Canada
1        United States
2        United States
3        United States
4               others
             ...      
48294           Canada
48295           Canada
48296    United States
48297        Australia
48298    United States
Name: User-Country, Length: 48299, dtype: object